In [ ]:
pip install transformers torch pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install XlsxWriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
import pandas as pd
import chardet
import string
import re
import statistics
from transformers import BertForQuestionAnswering, BertTokenizer, Trainer, TrainingArguments, default_data_collator
from datasets import Dataset, DatasetDict
import torch  # load pytorch
import torchvision
import xlsxwriter
from transformers import AutoTokenizer

In [ ]:
# Load Pre-trained BERT Model
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

# Define Paths
TEXT_FILES_DIR = "/content/DLData"
LABELS_CSV = "Item 5_Ans.csv"
OUTPUT_CSV = "extracted_answers.csv"

In [ ]:


## define the function for answering

def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    ## define special punctuations
    pun_left  = ['.', ')'] # punctuations that need to remove the left space
    pun_right = ['$', '('] # punctuations that need to remove the right space
    pun_both  = ['-', '/'] # punctuations that need to remove the both side spaces
    pun_special_1 = [','] # need to remove the left space, but if in a $ express, need to remove both side spaces.
    pun_special_2 = ['’'] # if follows by s, need to remove both side, otherwise, only need to remove left space
    pun_numeric = ['%', ',', '.']  # Ensure proper spacing for numeric values

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        elif tokens[i] in pun_left:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_both:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_special_1:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_special_2:
            answer += tokens[i] ## no space in the left side

        # Otherwise, add a space then the token.
        else:
            if i == 0:
                answer += tokens[i]
            else:
                if tokens[i-1] in pun_both:
                    answer +=  tokens[i]  ## no space in the left side of token[i]

                elif tokens[i-1] in pun_right:
                    answer +=  tokens[i]  ## no space in the left side of token[i]

                elif tokens[i-1] in pun_special_1:
                    if '$' in tokens[max(0, i-5):i]:  ## ',' in $ expression
                        answer +=  tokens[i]
                    else:
                        answer += ' ' + tokens[i]

                elif tokens[i-1] in pun_special_2:
                    if tokens[i] == 's':  ## ’s expression
                        answer +=  tokens[i]
                    else:
                        answer += ' ' + tokens[i]

                else:
                    answer += ' ' + tokens[i]

    return(answer)


In [ ]:

## define the function for computing probability

def answer_question_probability(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    ## convert tensor type to list, and then maximum it
    probability = max(max(start_scores.tolist())) + max(max(end_scores.tolist())) # they are scores, so add them together.

    return(probability)


In [ ]:

## define the function to compute the index of high probability for long file

def answer_question_longfile_probability(question, answer_text, max_num_sentences, overlap_num_sentences):

    ## First, split the answer text into sentences.
    content = answer_text.splitlines()

    ## get number of sentences
    len_sentence = len(content)

    ## how many pieces we will split, (get the ceiling of the division) + 1
    piece_num = - ( - (len_sentence -  max_num_sentences) // (max_num_sentences - overlap_num_sentences) ) + 1

    ## use prob to save all probilities
    prob = [0] * piece_num

    ##-------------------------------------------------##
    ## compute the probability of the first piece
    ##-------------------------------------------------##

    ## get the first piece
    content_temp = " "

    for ii in range(max_num_sentences):
        content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

    ## compute the probability for this piece (prob(start) * prob(end))
    prob[0] = answer_question_probability(question, content_temp)  # probability for the first piece

    ##------------------------------------------------------------##
    ## if only split into 2 pieces, go the last piece directly
    ##------------------------------------------------------------##

    if piece_num == 2:
        content_temp = " "

        for ii in range(max_num_sentences - overlap_num_sentences, len_sentence):
            content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

        ## compute the probability for this piece (prob(start) * prob(end))
        prob[1] = answer_question_probability(question, content_temp) # compute the probability for the last piece

    else: ## if split into more than 2 pieces

        for jj in range(1, piece_num - 1): ## compute pieces between the first one and the last one.

            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*jj, (max_num_sentences - overlap_num_sentences)*jj + max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## compute the probability for this piece (prob(start) * prob(end))
            prob[jj] = answer_question_probability(question, content_temp) # compute the probability for the last piece



        ## compute the last piece
        content_temp = " "

        for ii in range((max_num_sentences - overlap_num_sentences)*(piece_num - 1), len_sentence):
            content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

        ## compute the probability for this piece (prob(start) * prob(end))
        prob[piece_num - 1] = answer_question_probability(question, content_temp) # compute the probability for the last piece

    max_prob   = max(prob)  ## get the highese probability
    max_index  = prob.index(max_prob) ## find the index

    return(max_index)


In [ ]:
## define the function for splitting the files

def answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    ## if len(input_ids) < 512, this function goes to answer_question directly, otherwise we split the file into multiple sub-files.
    if len(input_ids) <= 512:

        answer = answer_question(question, answer_text)

       # print('Question: "' + question + '"')
       # print('Answer: "' + answer + '"')


    else:
        ## get the index of piece with the highest probability.
        max_index = answer_question_longfile_probability(question, answer_text, max_num_sentences, overlap_num_sentences)

        ##---------------------------------##
        ## get piece numbers               ##
        ##---------------------------------##
        ## First, split the answer text into sentences.
        content = answer_text.splitlines()

        ## get number of sentences
        len_sentence = len(content)

        ## how many pieces we will split, (get the ceiling of the division) + 1
        piece_num = - ( - (len_sentence -  max_num_sentences) // (max_num_sentences - overlap_num_sentences) ) + 1

        ## if it is the first piece
        if max_index == 0:
            ## get the piece
            content_temp = " "

            for ii in range(max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer

            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')


        ## if it is the last piece
        elif max_index == piece_num - 1:

            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*(piece_num - 1), len_sentence):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer
            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')

        else: # if in a middle piece
            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*max_index, (max_num_sentences - overlap_num_sentences)*max_index + max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer
            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')

    return(answer)


In [ ]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


In [ ]:
## this is a function to compute EM and F1 together

def compute_EM_F1(Prediction, Truth):
    # get how many questions
    len_para = len(Prediction)

    EM = ['']  # an empty list to contain the EM results
    F1 = ['']  # an emputy list to contain F1 results

    for ii in range(len_para):
        em_score = compute_exact_match(Prediction[ii], Truth[ii])  # compute em score one by one
        f1_score = compute_f1(Prediction[ii], Truth[ii])           # compute f1 score one by one

        EM.append(em_score) # add em score to EM
        F1.append(f1_score) # add f1 score to F1

    ## remove the first empty element in EM and F1
    EM = EM[1:len(EM)]
    F1 = F1[1:len(F1)]

    ## print the results out
    print(f"EM score: {statistics.mean(EM)}\n")
    print(f"F1 score: {statistics.mean(F1)}\n")

    return(EM, F1)

In [21]:
import pandas as pd
import os
import chardet

# Set up parameters and questions
Questions_item01 = Questions_item01 = ["How much is the initial franchise fee?",
                    "What is the initial franchise fee?",
                    "What is the initial fee?",
                    "How much fee do you pay at the time of franchise agreement?",
                    "How much is the standard initial franchise fee?",
                    "What is the amount of initial franchise fee?",
                    "What is the franchise fee?"]

max_num_sentences = 20
overlap_num_sentences = 5

# Create an empty list to store the answers for each file
all_answers = []
count =0
# Loop through all the files
for filename in os.listdir('/content/DLData'):
    if filename.endswith(".txt"):
        file_path = os.path.join('/content/DLData', filename)

        # Open the file and read the contents
        with open(file_path, "rb") as file:
            answer_text = file.read()
            encoding = chardet.detect(answer_text)['encoding']

        with open(file_path, "r", encoding=encoding) as file:
            answer_text = file.read()
        count +=1
        print(count)
        # Answer the questions for the current file
        current_answers = []
        for question in Questions_item01:
            answer = answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences)
            current_answers.append(answer)

        # Append the answers for the current file to the list
        all_answers.append(current_answers)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [22]:
# Get a list of processed file names
processed_files = [filename for filename in os.listdir('/content/DLData') if filename.endswith(".txt")]

# Create a pandas dataframe using the list of answers and the processed file names
df = pd.DataFrame(all_answers, columns=Questions_item01)  # Assign each question as a column
df.insert(0, "File Name", processed_files)  # Insert the processed file names at the beginning


In [23]:
import pandas as pd
from collections import Counter
import re
import string

# Load the ground truth answers
ground_truth_df = pd.read_csv("Item 5_Ans.csv")

# Merge the predicted answers with the ground truth answers based on the file name
merged_df = pd.merge(df, ground_truth_df, left_on="File Name", right_on="filename", how="inner")

def normalize_text(text):
    """Lowercase, remove punctuation, and extra whitespace."""
    text = text.lower().strip()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    return text

def compute_f1(pred, truth):
    """Compute F1 score between a predicted and ground truth answer."""
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()

    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    num_same = sum(common_tokens.values())

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0  # Avoid division by zero

    precision = num_same / len(pred_tokens)
    recall = num_same / len(truth_tokens)
    if precision + recall == 0:
        return 0

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def compute_EM_F1(predictions_list, truth, alternate_answers):
    """
    Compute EM and F1 scores given multiple predicted answers,
    the ground truth answer, and alternate answers.
    """
    truth_norm = normalize_text(truth)
    alt_norms = [(normalize_text(ans), ans) for ans in alternate_answers if pd.notna(ans)]

    final_predicted_answer = None

    for pred in predictions_list:
        if pd.isna(pred):
            continue
        pred_norm = normalize_text(pred)

        # Exact match with True Initial Fee
        if pred_norm == truth_norm:
            return 1, 1, pred

    # If no exact match with truth, check against alternate answers
    for alt_norm, alt_original in alt_norms:
        for pred in predictions_list:
            if pd.isna(pred):
                continue
            pred_norm = normalize_text(pred)
            if pred_norm == alt_norm:
                f1 = compute_f1(pred, alt_original)  # Compute F1 with matched alternative
                return 0.5, f1, pred

    # No match at all
    return 0, 0, None

# Iterate through merged DataFrame and compute EM/F1 scores
exact_matches = []
f1_scores = []
final_answers = []

for _, row in merged_df.iterrows():
    predictions = [
        row["How much is the initial franchise fee?"],
        row["What is the initial franchise fee?"],
        row["What is the initial fee?"],
        row["How much fee do you pay at the time of franchise agreement?"],
        row["How much is the standard initial franchise fee?"],
        row["What is the amount of initial franchise fee?"],
        row["What is the franchise fee?"]
    ]

    truth = row["True Initial Fee"]
    alternate_answers = [
        row["Alternative Answer 1"],
        row["Alternative Answer 2"],
        row["Alternative Answer 3"],
        row["Alternative Answer 4"],
        row["Alternative Answer 5"]
    ]

    em, f1, final_pred = compute_EM_F1(predictions, truth, alternate_answers)
    exact_matches.append(em)
    f1_scores.append(f1)
    final_answers.append(final_pred if final_pred else "No Match")

# Store EM, F1 scores, and final predicted answers in the dataframe
merged_df["Exact Match"] = exact_matches
merged_df["F1 Score"] = f1_scores
merged_df["Final_predicted_answer"] = final_answers

# Compute overall EM and F1 scores
overall_em = sum(exact_matches) / len(exact_matches)
overall_f1 = sum(f1_scores) / len(f1_scores)

# Print overall scores
print(f"Overall Exact Match (EM): {overall_em:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")


Overall Exact Match (EM): 0.8900
Overall F1 Score: 0.9200


In [24]:

# Extract relevant columns and export to CSV
extracted_df = merged_df[['File Name', 'True Initial Fee', 'Alternative Answer 1', 'Alternative Answer 2', 'Alternative Answer 3', 'Alternative Answer 4', 'Alternative Answer 5', 'Exact Match', 'F1 Score', 'Final_predicted_answer']]
extracted_df.to_csv('extracted_answers.csv', index=False)
